In [1]:
#import required librairies
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier,RandomForestClassifier 
from sklearn.neural_network import MLPClassifier

In [3]:
#read both train and test datasets
ds1=pd.read_csv('C://Users/user/Downloads/train.csv')
ds2=pd.read_csv('C://Users/user/Downloads/test.csv')

In [4]:
#remove unnecessery columns and create a lable vector from train data(ds1)
X_train=ds1.drop(labels=['default payment next month','Unnamed: 0','ID'],axis=1)
X_train=X_train.rename(columns={'PAY_0':'PAY_1'})
y_train=ds1['default payment next month']

X_test=ds2.drop(labels=['Unnamed: 0','ID'],axis=1)
X_test=X_test.rename(columns={'PAY_0':'PAY_1'})

In [5]:
#categorie given money amount from o to 8 for every 100000 
for dataset in [X_train,X_test]:
    cols=['LIMIT_BAL','BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']
    for col in cols:
        dataset.loc[(dataset[col] <= 100000),col] = 0
        dataset.loc[(dataset[col] > 100000) & (dataset[col] <= 200000),col] = 1
        dataset.loc[(dataset[col] > 200000) & (dataset[col] <= 300000),col] = 2
        dataset.loc[(dataset[col] > 300000) & (dataset[col] <= 400000),col] = 3
        dataset.loc[(dataset[col] > 400000) & (dataset[col] <= 500000),col] = 4
        dataset.loc[(dataset[col] > 500000) & (dataset[col] <= 600000),col] = 5
        dataset.loc[(dataset[col] > 600000) & (dataset[col] <= 700000),col] = 6
        dataset.loc[(dataset[col] > 700000) & (dataset[col] <= 800000),col] = 7
        dataset.loc[(dataset[col] > 800000),col] = 8

In [6]:
#Initialize RandomForestClassifier, KNeighborsClassifier, GaussianNB, DescisionTreeClassifier, Multi-Layer-Perceptron 
rf = RandomForestClassifier(criterion = "gini", 
                                       min_samples_leaf = 1, 
                                       min_samples_split = 10,   
                                       n_estimators=100, 
                                       max_features='auto', 
                                       oob_score=True, 
                                       random_state=1, 
                                       n_jobs=-1)
rf.fit(X_train,y_train)
knn = KNeighborsClassifier(n_neighbors = 9,n_jobs=-1,weights='uniform',algorithm='auto')
knn.fit(X_train,y_train)
gnb = GaussianNB(priors=None, var_smoothing=1e-09)
gnb.fit(X_train,y_train)
dt=DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=5, min_samples_split=2, min_samples_leaf=1)
gnb.fit(X_train,y_train)
mlpc=MLPClassifier(hidden_layer_sizes=(25,25,25),max_iter=500)
mlpc.fit(X_train,y_train)

#initialise VotingClassifier from all above estimators

eclf = VotingClassifier(estimators=[('knn', knn), ('rf', rf), ('gnb', gnb),('dt',dt),('mlpc',mlpc)], voting='hard')
eclf.fit(X_train,y_train)

VotingClassifier(estimators=[('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=9, p=2,
           weights='uniform')), ('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, ma...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [7]:
#predict label values for test data from trained model
y_pred=eclf.predict(X_test)

In [8]:
ds2['Prediction']=y_pred

In [9]:
ds2.to_csv('C://Users/user/Downloads/test.csv')